# MNIST 模型強化

## 步驟1：載入 MNIST 手寫阿拉伯數字資料

In [1]:
import tensorflow as tf
mnist = tf.keras.datasets.mnist

# 載入 MNIST 手寫阿拉伯數字資料
(x_train, y_train),(x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


## 步驟2：改用 CNN 模型

In [2]:
# 建立模型
from tensorflow.keras import layers
import numpy as np

input_shape=(28, 28, 1)
# 增加一維在最後面
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# CNN 模型
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation="softmax"),
    ]
)

# 設定優化器(optimizer)、損失函數(loss)、效能衡量指標(metrics)的類別
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## 步驟3：資料增補(Data Augmentation)

In [3]:
# 參數設定
batch_size = 1000
epochs = 5

# 資料增補定義
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,         # 特徵縮放
        rotation_range=10,      # 旋轉 10 度
        zoom_range=0.1,         # 拉遠/拉近 10%
        width_shift_range=0.1,  # 寬度偏移  10%
        height_shift_range=0.1) # 高度偏移  10%

# 增補資料，進行模型訓練
datagen.fit(x_train)
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
          validation_data=datagen.flow(x_test, y_test, batch_size=batch_size), verbose=2,
          steps_per_epoch=x_train.shape[0]//batch_size)

Epoch 1/5
60/60 - 36s - loss: 1.4183 - accuracy: 0.5463 - val_loss: 0.6056 - val_accuracy: 0.8237 - 36s/epoch - 602ms/step
Epoch 2/5
60/60 - 21s - loss: 0.5388 - accuracy: 0.8292 - val_loss: 0.3207 - val_accuracy: 0.9062 - 21s/epoch - 352ms/step
Epoch 3/5
60/60 - 21s - loss: 0.3752 - accuracy: 0.8845 - val_loss: 0.2395 - val_accuracy: 0.9290 - 21s/epoch - 347ms/step
Epoch 4/5
60/60 - 21s - loss: 0.3049 - accuracy: 0.9071 - val_loss: 0.1978 - val_accuracy: 0.9426 - 21s/epoch - 355ms/step
Epoch 5/5
60/60 - 23s - loss: 0.2619 - accuracy: 0.9191 - val_loss: 0.1651 - val_accuracy: 0.9509 - 23s/epoch - 377ms/step


In [4]:
# 評分(Score Model)
score=model.evaluate(x_test, y_test, verbose=0)

for i, x in enumerate(score):
    print(f'{model.metrics_names[i]}: {score[i]:.4f}')

loss: 11.6659
accuracy: 0.9717


## 步驟4：測試自行繪製的數字

In [8]:
# 使用小畫家，繪製 0~9，實際測試看看
from skimage import io
from skimage.transform import resize
import numpy as np

# 讀取影像並轉為單色
uploaded_file = './myDigits/a8.png'
image1 = io.imread(uploaded_file, as_gray=True)

# 縮為 (28, 28) 大小的影像
image_resized = resize(image1, (28, 28), anti_aliasing=True)    
X1 = image_resized.reshape(1,28, 28, 1) #/ 255

# 反轉顏色，顏色0為白色，與 RGB 色碼不同，它的 0 為黑色
X1 = np.abs(1-X1)

# 預測
predictions = np.argmax(model.predict(X1), axis=-1)
print(predictions)

1/1 [==============================] - 0s 20ms/step
[8]
